# Running a Grid

A common task is to run UCLCHEM over a grid of parameter combinations. This notebook sets up a simple approach to doing so for regular grids.

In [1]:
import uclchem
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed

## A Simple Grid
### Define Parameter Space
First, we define our parameter space. We do this by using numpy and pandas to produce a table of all possible combinations of some parameters of interest.

In [2]:
# This part can be substituted with any choice of grid
# here we just vary the density, temperature and zeta
temperatures = np.linspace(10, 50, 3)
densities = np.logspace(4, 6, 3)
zetas = np.logspace(1, 3, 3)

# meshgrid will give all combinations, then we shape into columns and put into a table
parameterSpace = np.asarray(np.meshgrid(temperatures, densities, zetas)).reshape(3, -1)
model_table = pd.DataFrame(parameterSpace.T, columns=["temperature", "density", "zeta"])

# keep track of where each model output will be saved and make sure that folder exists
model_table["outputFile"] = model_table.apply(
    lambda row: f"../grid_folder/{row.temperature}_{row.density}_{row.zeta}.csv", axis=1
)
print(f"{model_table.shape[0]} models to run")
if not os.path.exists("../grid_folder"):
    os.makedirs("../grid_folder")

27 models to run


In [3]:
model_table.head()

,temperature,density,zeta,outputFile
0,10.0,10000.0,10.0,../grid_folder/10.0_10000.0_10.0.csv
1,10.0,10000.0,100.0,../grid_folder/10.0_10000.0_100.0.csv
2,10.0,10000.0,1000.0,../grid_folder/10.0_10000.0_1000.0.csv
3,30.0,10000.0,10.0,../grid_folder/30.0_10000.0_10.0.csv
4,30.0,10000.0,100.0,../grid_folder/30.0_10000.0_100.0.csv


### Set up the model
Next, we need a function that will run our model. We write a quick function that takes a row from our dataframe and uses it to populate a parameter dictionary for UCLCHEM and then run a cloud model. We can then map our dataframe to that function.

In [4]:
def run_model(row):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": False,
        "freefall": False,
        "initialDens": row.density,
        "initialTemp": row.temperature,
        "zeta": row.zeta,
        "outputFile": row.outputFile,
        "finalTime": 1.0e6,
        "baseAv": 10,
        "abstol_min": 1e-22,
    }
    result = uclchem.model.Cloud(param_dict=ParameterDictionary)
    return result.success_flag  # just the integer error code

### Run Grid 

#### The Simple Way
We can use pandas apply to simply pass each row to our helper function in turn. This will take some time since we're running the models one by one. I'll use the `head` function just to run five rows as an example here.

In [5]:
%%time
result = model_table.head().apply(run_model, axis=1)

CPU times: user 8.34 s, sys: 27 ms, total: 8.37 s
Wall time: 8.34 s


#### The Fast Way
Alternatively, we can use multiprocessing to run the models in parallel. That will allow us to run many models simulataneously and make use of all the cores available on our machine.

In [6]:
%%time
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_model)(row) for idx, row in model_table.iterrows()
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    9.3s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    9.8s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   12.4s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:   14.0s
[Parallel(

## Checking Your Grid
After running, we should do two things. First, let's add `results` to our dataframe as a new column. Positive results mean a successful UCLCHEM run and negative ones are unsuccessful. Then we can run each model through `check_element_conservation` to check the integration was successful. We'll use both these things to flag models that failed in some way.

In [7]:
def element_check(output_file):
    df = uclchem.analysis.read_output_file(output_file)
    # get conservation values
    conserves = uclchem.analysis.check_element_conservation(df)
    # check if any error is greater than 1%
    return all([float(x[:-1]) < 1 for x in conserves.values()])

In [8]:
model_table["run_result"] = results
model_table["elements_conserved"] = model_table["outputFile"].map(element_check)
# check both conditions are met
model_table["Successful"] = (model_table.run_result >= 0) & (
    model_table.elements_conserved
)

In [9]:
model_table.head()

,temperature,density,zeta,outputFile,run_result,elements_conserved,Successful
0,10.0,10000.0,10.0,../grid_folder/10.0_10000.0_10.0.csv,0,True,True
1,10.0,10000.0,100.0,../grid_folder/10.0_10000.0_100.0.csv,0,True,True
2,10.0,10000.0,1000.0,../grid_folder/10.0_10000.0_1000.0.csv,0,True,True
3,30.0,10000.0,10.0,../grid_folder/30.0_10000.0_10.0.csv,0,True,True
4,30.0,10000.0,100.0,../grid_folder/30.0_10000.0_100.0.csv,0,True,True


## Complex Grid

The above was straightforward enough but what about a modelling a grid of shocks? Not only do we want to loop over relevant parameters, we also need to run a few preliminary models to give ourselves starting abundances. We'll start by defining two helper functions, one to run our preliminary cloud and one to run the shock.

Let's further imagine that we want to obtain the abundances of several species at the end of the model. We can use the `out_species` parameter to specify which species we want to track and return them to our dataframe.

In [10]:
out_species = ["CO", "H2O", "CH3OH"]


def run_prelim(density):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": True,
        "freefall": True,
        "initialDens": 1e2,
        "finalDens": float(density),
        "initialTemp": 10.0,
        "abundSaveFile": f"../grid_folder/starts/{density:.0f}.csv",
        "baseAv": 1,
    }
    result = uclchem.model.Cloud(param_dict=ParameterDictionary)
    return result.success_flag


def run_model(row):
    # basic set of parameters we'll use for this grid.
    ParameterDictionary = {
        "endatfinaldensity": False,
        "freefall": False,
        "initialDens": float(row.density),
        "initialTemp": 10.0,
        "outputFile": row.outputFile,
        "abundLoadFile": f"../grid_folder/starts/{row.density:.0f}.csv",
        "finalTime": 1.0e5,
        "abstol_factor": 1e-14,
        "abstol_min": 1e-20,
        "reltol": 1e-6,
        "baseAv": 1,
    }
    result = uclchem.model.CShock(
        row.shock_velocity, param_dict=ParameterDictionary, out_species=out_species
    )
    # First check UCLCHEM's result flag to seeif it ran succesfully, if it is return the abundances
    if result.success_flag == 0:
        return result.success_flag, result.dissipation_time, result.chemical_abun_array
    # if not, return NaNs because model failed
    else:
        return [np.nan] * len(out_species)

Then we define our parameter space again. We'll create two folders, one to store a set of initial abundances for each starting density in our model and another to store our shock outputs.

In [11]:
# This part can be substituted with any choice of grid
# here we just combine various initial and final densities into an easily iterable array
shock_velocities = np.linspace(10, 50, 3)
densities = np.logspace(4, 6, 3)

parameterSpace = np.asarray(np.meshgrid(shock_velocities, densities)).reshape(2, -1)
model_table = pd.DataFrame(parameterSpace.T, columns=["shock_velocity", "density"])
model_table["outputFile"] = model_table.apply(
    lambda row: f"../grid_folder/shocks/{row.shock_velocity}_{row.density}.csv", axis=1
)
print(f"{model_table.shape[0]} models to run")

for folder in ["starts", "shocks"]:
    if not os.path.exists(f"../grid_folder/{folder}"):
        os.makedirs(f"../grid_folder/{folder}")

9 models to run


We can then run our preliminary models followed by our science models. The science models will return the abundances at the final time step of each run so we can unpack those directly to our dataframe.

In [12]:
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_prelim)(dens) for dens in densities
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    4.2s finished


In [ ]:
results = Parallel(n_jobs=4, verbose=100)(
    delayed(run_model)(row) for idx, row in model_table.iterrows()
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
 At current T(=R1), MXSTEP(=I1) steps                                            
 taken on this call before reaching TOUT.                                        
In the above message, I1 =      10000
In the above message, R1 =   0.5369341152504D+09
 ISTATE -1: Reducing time step to    5.8689509575692758E-002 years
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    8.1s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    8.3s
[Parallel(n_jobs=4)]: Done   3 out of   9 | elapsed:   13.2s remaining:   26.5s
 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.1341010698693D+13   R2 =   0.3314263159790D+04
 ISTATE -5 - shortening step at time   40484.767492356004      years
 At T(=R1) a

In [18]:
model_table[["Result", "Dissipation Time"] + out_species] = results

In [19]:
model_table

,shock_velocity,density,outputFile,Result,Dissipation Time,CO,H2O,CH3OH
0,10.0,10000.0,../grid_folder/shocks/10.0_10000.0.csv,0.0,1171.898734,5.973692e-05,2.390674e-05,2.573049e-11
1,30.0,10000.0,../grid_folder/shocks/30.0_10000.0.csv,0.0,1171.898734,2.502361e-05,2.420695e-05,9.976796e-07
2,50.0,10000.0,../grid_folder/shocks/50.0_10000.0.csv,0.0,1171.898734,1.339461e-05,1.069392e-05,5.981371e-07
3,10.0,100000.0,../grid_folder/shocks/10.0_100000.0.csv,0.0,117.189873,1.233292e-07,1.140102e-09,1.453093e-11
4,30.0,100000.0,../grid_folder/shocks/30.0_100000.0.csv,0.0,117.189873,1.146282e-22,1.279723e-20,1.181856e-20
5,50.0,100000.0,../grid_folder/shocks/50.0_100000.0.csv,0.0,117.189873,1.252981e-23,1.076729e-21,1.259819e-21
6,10.0,1000000.0,../grid_folder/shocks/10.0_1000000.0.csv,0.0,11.718987,3.743088e-27,6.926607e-23,1.151194e-22
7,30.0,1000000.0,../grid_folder/shocks/30.0_1000000.0.csv,0.0,11.718987,2.389471e-29,1.420828e-23,1.372137e-23
8,50.0,1000000.0,../grid_folder/shocks/50.0_1000000.0.csv,0.0,11.718987,4.851176e-29,2.837123e-23,3.136032e-23


## Summary

There are many ways to run grids of models and users will naturally develop their own methods. This notebook is just a simple example of how to run UCLCHEM for many parameter combinations whilst producing a useful output (the model_table) to keep track of all the combinations that were run. In a real script, we'd save the model file to csv at the end.

For much larger grids, it's recommended that you find a way to make your script robust to failure. Over a huge range of parameters, it is quite likely UCLCHEM will hit integration trouble for at least a few parameter combinations. Very occasionally, UCLCHEM will get caught in a loop where it fails to integrate and cannot adjust its strategy to manage it. This isn't a problem for small grids as the model can be stopped and the tolerances adjusted. However, for very large grids, you may end up locking all threads as they each get stuck on a different model. The best solution we've found for this case is to add a check so that models in your dataframe are skipped if their file already exists, this allows you to stop and restart the grid script as needed.
